In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint, History, CallbackList, BaseLogger
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
from fastmri_recon.data.test_generators import RandomShapeGenerator
from fastmri_recon.helpers.adversarial_training import compile_models, adversarial_training_loop
from fastmri_recon.helpers.image_tboard_cback import TensorBoardImage
from fastmri_recon.helpers.keras_utils import wasserstein_loss
from fastmri_recon.models.discriminator import discriminator_model, generator_containing_discriminator_multiple_outputs
from fastmri_recon.models.unet import unet

In [3]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
}

im_size = 64

def generator_model():
    model = unet(input_size=(im_size, im_size, 1), **run_params, compile=False)
    model.name = 'Reconstructor'
    return model

In [4]:
# model definitions
g = generator_model()
# prev_run_id = 'unet_wo_lastrelu_af4_1572013433'
# prev_epoch = 500
# chkpt_path = f'checkpoints/{prev_run_id}-{prev_epoch}.hdf5'
# g.load_weights(chkpt_path)
d = discriminator_model(im_size)
d_on_g = generator_containing_discriminator_multiple_outputs(g, d, im_size=im_size)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [5]:
# model compiling
perceptual_loss = 'mae'

discriminator_lr = 1e-4
d_on_reconstructor_lr = 1e-3

compile_models(
    d, 
    g, 
    d_on_g, 
    d_lr=discriminator_lr, 
    d_on_g_lr=d_on_reconstructor_lr, 
    perceptual_loss=perceptual_loss, 
    perceptual_weight=1.0,
)

In [6]:
n_batches_train = 1000
n_batches_val = 1
AF = 2
train_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()
val_gen = RandomShapeGenerator(af=AF, n_shapes=100, size=im_size, batch_size=1).flow_z_filled_random_shapes()

epoch_num = 100
critic_updates = 5

run_id = f'unet_gan_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_gan_af2_1573655269


In [7]:
log_dir = op.join('logs/gan', run_id)

tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    write_graph=True, 
    update_freq=50,
)

chkpt_cback = ModelCheckpoint(chkpt_path, period=epoch_num)
selected_slice = 0
data = next(val_gen)
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=data[1][selected_slice:selected_slice+1],
    # NOTE: for cross-domain slice has to be on kspace and mask
    model_input=data[0][selected_slice:selected_slice+1],
)

log_dir_pre = op.join('logs/pregan', run_id)
gen_tensor = TensorBoard(
    log_dir=log_dir_pre
)
gen_tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

/Users/WorkAccount/miniconda3/envs/gan/lib/python3.7/site-packages/fastmri_recon-0.0.1-py3.7.egg/fastmri_recon/data/test_generators.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
/Users/WorkAccount/miniconda3/envs/gan/lib/python3.7/site-packages/fastmri_recon-0.0.1-py3.7.egg/fastmri_recon/helpers/reconstruction.py:40: ComplexWarning: Casting complex values to real discards the imaginary part


In [8]:
%%time
# %debug
# overfitting trial
adversarial_training_loop(
    g, 
    d, 
    d_on_g, 
    train_gen, 
    n_epochs=100, 
    n_batches=1, 
    n_critic_updates=5,
    callbacks=[tqdm_cb, tboard_cback, tboard_image_cback],
    include_d_metrics=True,
    gen_pre_training_steps=100,
    pre_training_callbacks=[gen_tensor,gen_tqdm_cb]
)

/Users/WorkAccount/miniconda3/envs/gan/lib/python3.7/site-packages/fastmri_recon-0.0.1-py3.7.egg/fastmri_recon/data/test_generators.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
/Users/WorkAccount/miniconda3/envs/gan/lib/python3.7/site-packages/fastmri_recon-0.0.1-py3.7.egg/fastmri_recon/helpers/reconstruction.py:40: ComplexWarning: Casting complex values to real discards the imaginary part



CPU times: user 2min 7s, sys: 51.1 s, total: 2min 58s
Wall time: 1min 42s
